In [4]:
import pandas as pd
import os
import numpy as np


In [2]:
df = pd.read_csv(os.path.join(os.getcwd(), '../resources/accuracy_set.csv'), index_col=0, dtype={'input' : object, 'output': object})

drop_columns = ['Unnamed: 0', 'SynBioHub URI', 'Unnamed: 0.1', 'calibration_id', 'experiment_id', 'filename', 
                'glycerol_plate_index',  'growth_media_1', 'growth_media_2',  
                'inc_temp', 'inc_time_1', 'inc_time_2', 'input', 'lab', 'lab_id', 'media', 'od', 
                'od_cutoff', 'output', 'part_1_id', 'part_2_id', 'plate_id',  'replicate', 'source_container', 
                'strain_name', 'strain_circuit', 'well', 'post_well']
od_df = pd.read_csv(os.path.join(os.getcwd(), '../resources/tx_od.csv')).rename(columns={'strain' : 'strain_name'}).drop(columns=drop_columns)
df = df.merge(od_df, on='id', how='left')



In [15]:
correctness_col='mean_correct_classifier_live'
conditions=['inc_time_2', 'media', 'inc_temp', 'od']
design=['gate', 'input']


# Strain x condition scores is mean of samples for condition
strain_score_df = df.groupby(conditions+design)[correctness_col].agg(np.mean).reset_index()

# Circuit x condition score is mean of strains comprising the circuit
circuit_score_df = strain_score_df.groupby(conditions+['gate']).agg(np.mean).reset_index()

# Robustness is the std deviation of the score across all conditions
circuit_robustness_df = circuit_score_df.groupby(['gate'])[correctness_col].agg(np.std).reset_index()
circuit_robustness_df.loc[:, 'robustness'] = circuit_robustness_df.apply(lambda x: 1.0- x[correctness_col], axis=1)
circuit_robustness_df = circuit_robustness_df.drop(columns=[correctness_col])
circuit_robustness_df

,gate,robustness
0,AND,0.913390
1,NAND,0.784127
2,NOR,0.811018
3,OR,0.806068
4,XNOR,0.889020
5,XOR,0.818948
